# Dog Breed Identification

**Overview:**
- Summarizing 'What I've learnt'?
- Prepare Data
- Inspect Data
- Building, Training and Evaluating Models
- Submission
- Notes and Conclusion

### What I've learnt?
- Concepts:
- Code:

**Imports:**

In [1]:
import os, shutil

import pandas as pd
import numpy as np
import cv2

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.applications import VGG16

import matplotlib.pyplot as plt
%matplotlib inline

# custom imports
from scripts import my_utils

%reload_ext autoreload
%autoreload 2

C:\Users\Jagan\Anaconda3\envs\deep-learning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Preparing Data:

In [29]:
src_dir="C:\\Users\Jagan\.kaggle\competitions\dog-breed-identification"
base_dir = '..\datasets\dog_breed_identification'

In [30]:
my_utils.unpack_dataset(src_dir, base_dir)

Tests passed.
Extracting from 'C:\Users\Jagan\.kaggle\competitions\dog-breed-identification' to '..\datasets\dog_breed_identification'.

This might take a while...
Finished. Time taken: 41.29s.


In [41]:
mapping = pd.read_csv(os.path.join(base_dir,'labels.csv'))
train_dir, val_dir = my_utils.bin_dataset(base_dir, 'train', mapping.values, validation_split = 0.2)
test_dir = os.path.join( base_dir, 'test')

Tests passed.
Total no. of files: 
Selected no. of files:  10222

This might take a while.... No. of train samples: 8177
No. of validation samples: 2045
Done. Time taken: 120.49s.


In [ ]:
# directory to save models
save_dir = '..\saved_models\dog_breed_identification'
if not os.path.isdir(save_dir): os.makedirs(save_dir)

**Data Parameters:**

In [ ]:
img_rows, img_cols, img_chnls = 150, 150, 3
input_shape = (img_rows,img_cols,img_chnls)
target_size = input_shape[:-1]

num_classes = np.unique(mapping.values[:,1]).shape[0]

train_sample_count = 8177 
val_sample_count = 2045
test_sample_count = len(os.listdir(test_dir))
batch_size = 256

### Inspecting Data:

### Building, Training and Evaluating Models:

**1. Basic Model:** 

In [ ]:
# model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# compile model
model.compile(optimizer = RMSprop(lr = 1e-4), 
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

In [ ]:
# data generators
train_generator = my_utils.data_generator(train_dir, target_size, batch_size)
val_generator = my_utils.data_generator(val_dir, target_size, batch_size)

In [ ]:
# fitting parameters
epochs = 1

In [ ]:
# training
history = model.fit_generator(
                  train_generator,
                  steps_per_epoch = train_sample_count // batch_size,
                  epochs = epochs,
                  validation_data = val_generator,
                  validation_steps = val_sample_count // batch_size)

In [ ]:
# save model
model_name = 'dbi_basic_1.h5'
model.save(os.path.join(save_dir, model_name))

In [ ]:
# plot model history
my_utils.plot_history(history)

**2. Basic Model with Data Augmentation:** 

In [ ]:
# model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# compile model
model.compile(optimizer = RMSprop(lr = 1e-4), 
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

In [ ]:
# data generators
train_generator = my_utils.data_generator(train_dir, target_size, batch_size, augment = True)
val_generator = my_utils.data_generator(val_dir, target_size, batch_size)

In [ ]:
# fitting parameters
epochs = 1

In [ ]:
# training
history = model.fit_generator(
                  train_generator,
                  steps_per_epoch = train_sample_count // batch_size,
                  epochs = epochs,
                  validation_data = val_generator,
                  validation_steps = val_sample_count // batch_size)

In [ ]:
# save moodel
model_name = 'dbi_basic_data_aug_2.h5'
model.save(os.path.join(save_dir, model_name))

In [ ]:
# plot model history
my_utils.plot_history(history)

In [ ]:
# evaluating on test data
test_generator = my_utils.data_generator(test_dir, target_size, batch_size)
predictions = model.predict_generator(test_generator, 
                                      steps=None, 
                                      max_queue_size=10, 
                                      workers=1, 
                                      use_multiprocessing=False, 
                                      verbose=0)

predictions = np.argmax(predictions, axis=-1) #multiple categories

label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

**3. Using Pre-Trained ConvNet:** 

In [ ]:
# vgg16 convbase
vgg16_conv_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
vgg16_conv_base.summary()

From here on there's two ways to go about this:
- **Fast Feature Extraction:** Running the convolutional base over our dataset, recording its output to a Numpy array on disk, then using this data as input to a standalone densely-connected classifier. This solution is very fast and cheap to run, because it only requires running the convolutional base once for every input image, and the convolutional base is by far the most expensive part of the pipeline. However, for the exact same reason, this technique would not allow us to leverage data augmentation at all.
- **Model Extension:** Extending the model we have (conv_base) by adding Dense layers on top, and running the whole thing end-to-end on the input data. This allows us to use data augmentation, because every input image is going through the convolutional base every time it is seen by the model. However, for this same reason, this technique is far more expensive than the first one.

**Fast Feature Extraction:**

In [ ]:
# input shape for dense model
feature_shape = (vgg16_conv_base.layers[-1]).output_shape
dm_input_shape = np.prod(feature_shape[1:])

In [ ]:
# dense model
model = Sequential()
    
model.add(Dense(512, activation='relu', input_shape=input_shape))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

dense_model.summary()

In [ ]:
# compile modle
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
# data generators
train_generator = my_utils.data_generator(train_dir, target_size, batch_size)
val_generator = my_utils.data_generator(val_dir, target_size, batch_size)

In [ ]:
# extract features
train_features, train_labels = my_utils.extract_features(vgg16_conv_base, train_sample_count, 
                                                           train_generator, num_classes, batch_size)
val_features, val_labels = my_utils.extract_features(vgg16_conv_base, val_sample_count, 
                                                       val_generator, num_classes, batch_size)

In [ ]:
# reshape extracted features  
train_features = train_features.reshape(train_sample_count, -1)
val_features = val_features.reshape(val_sample_count, -1)

In [ ]:
# fitting parameters
epochs = 1

In [ ]:
# training
history = model.fit(train_features, train_labels,
                    epochs = epochs,
                    batch_size = batch_size,
                    validation_data = (validation_features, validation_labels))

In [ ]:
# save moodel
model_name = 'dbi_pre_trained_fast_feature_extraction_3.h5'
model.save(os.path.join(save_dir, model_name))

In [ ]:
# plot model history
my_utils.plot_history(history)

**Model Extension:**

In [ ]:
# model
model = Sequential()

model.add(vgg16_conv_base)

model.add(Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# freeze the conv_base
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))
vgg16_conv_base.trainable = False
print('This is the number of trainable weights after freezing the conv base:', len(model.trainable_weights))

In [ ]:
# compile modle
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
# data generators
train_generator = my_utils.data_generator(train_dir, target_size, batch_size, augment = True)
val_generator = my_utils.data_generator(val_dir, target_size, batch_size)

In [ ]:
# fitting parameters
epochs = 1

In [ ]:
# training
history = model.fit_generator(
                  train_generator,
                  steps_per_epoch = train_sample_count // batch_size,
                  epochs = epochs,
                  validation_data = val_generator,
                  validation_steps = val_sample_count // batch_size)

In [ ]:
# save moodel
model_name = 'dbi_pre_trained_model_extension_4.h5'
model.save(os.path.join(save_dir, model_name))

In [ ]:
# plot model history
my_utils.plot_history(history)

**Fine-tuning:** 

In [ ]:
# freezing everything but the last block
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
# fitting parameters
epochs = 1

In [ ]:
# training
history = model.fit_generator(
                  train_generator,
                  steps_per_epoch = train_sample_count // batch_size,
                  epochs = epochs,
                  validation_data = val_generator,
                  validation_steps = val_sample_count // batch_size)

In [ ]:
# save moodel
model_name = 'dbi_fine_tuning_5.h5'
model.save(os.path.join(save_dir, model_name))

In [ ]:
# plot model history
my_utils.plot_history(history)